朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布 $P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：
$$P(X,Y)＝P(Y)P(X|Y)$$
概率估计方法可以是极大似然估计或贝叶斯估计。
2．朴素贝叶斯法的基本假设是条件独立性，
$$\begin{aligned} P(X&amp;=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &amp;=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$
这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。
3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。
$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
将输入$x$分到后验概率最大的类$y$。
$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$
后验概率最大等价于0-1损失函数时的期望风险最小化。
模型：
高斯模型
多项式模型
伯努利模型


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math


In [2]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100, :])
    # print(data)
    return data[:, :-1], data[:, -1]


In [3]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [4]:
X_test[0], y_test[0]


(array([5. , 3.4, 1.6, 0.4]), 0.0)

特征的可能性被假设为高斯

概率密度函数： $$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$


In [23]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
array = np.array(x)
print(array)
xyz = zip(x, y, z)
# u = zip(*xyz)
# print(list(xyz))
print(i for i in zip(*xyz))
# zip两个重点：一是储存zip函数结果的变量内存在使用过一次后就被释放
# 二是，zip是147，258这种按列来组合，再zip(*)后就变成了123，456，按行结合
# 本质上就是zip，按列组成一个迭代对象，zip(*)按行组成一个迭代对象，迭代就是x、y、z

[1 2 3]
<generator object <genexpr> at 0x7fdfd09867d0>


In [5]:
class NaiveBayes(object):
    def __init__(self):
        self.model = None

    # 每一个特征的期望
    @staticmethod
    def mean(x):
        return sum(x)/float(len(x))
    # 方差，是sigma不是sigma平方
    def stdev(self, x):
        avg = self.mean(x)
        # 和下面区分，中括号里面是pow，小括号就是math.pow
        return math.sqrt(sum([pow(x1 - avg, 2) for x1 in x]) / float(len(x)))

    def gaussian_probability(self, x, mean, stdev):
        # 和上面区分写法，注意这里是对应的某个特征的高斯分布
        # 其中的输入变量为
        exponent = math.exp(-(math.pow(x - mean, 2))
                            / 2*(math.pow(stdev, 2)))
        return (1/math.sqrt(2 * math.pi *stdev)) * exponent

        # 处理X_train
    def summarize(self, train_data):
        # 对代表特征的列，一列一列的求期望和方差
        # 求每一行代表的数据的期望和方差
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries

        # 分类别求出数学期望和标准差
    def fit(self, X, y):
        # 构建集合的时候重复元素被合并了
        labels = list(set(y))
        # [0.0, 1.0]
        data = {label: [] for label in labels}
        # {0.0:[], 1.0:[]}
        for f, label in zip(X, y):
            data[label].append(f)
            #相当于对键为label的后面把f加上去
            # {0.0: [array([5.1, 3.5, 1.4, 0.2]), array([4.9, 3. , 1.4, 0.2]), array([4.7, 3.2, 1.3, 0.2]), array([4.6, 3.1, 1.5, 0.2]).....
        self.model = {
            label: self.summarize(value)
            # items()是字典的方法，把键和值递归的取出来，这里取出的value是一个100*4的数组
            for label, value in data.items()
        }
        # model存储是一个字典，为{label:[(mean， stdev), (mean， stdev), (mean， stdev)...], label:...}
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            # {0.0:1, 1.0:1}
            for i in range(len(value)):
                # 从里面取出一个数据的mean和std
                mean, stdev = value[i]
                # 循环与每一个数据比较，得到对应于每一个类别的条件概率的所有数据的连积
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)
        return probabilities

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        # 按照字典每个数据的第二个值来进行降序排列，最后一个为概率最大的，取出其键即为所求
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [6]:
model = {0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
probabilities = {}
clf = NaiveBayes()
for label, value in model.items():
    probabilities[label] = 1
    for i in range(len(value)):
        mean, stdev = value[i]
        probabilities[label] *=clf.gaussian_probability(
                    X_train[i], mean, stdev)

TypeError: only size-1 arrays can be converted to Python scalars

In [7]:
model = NaiveBayes()


In [8]:
model.fit(X_train, y_train)


'gaussianNB train done!'

In [9]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))


0.0


In [10]:
model.score(X_test, y_test)


0.5333333333333333